# Introduction to Database Design
---------------------------------------------------------------------------------------------------------

In our previous session on databases, we introduced some of the fundamental concepts and definitions applicable to databases in general, along with a brief intro to SQL and SQLite in particular. Some use cases and platforms were also discussed.

In this session, we are going to dig a little deeper into databases as representions of systems and processes. A database with a single table may not feel or function much differently from a spreadsheet. Much of the benefit of using databases results from designing them as models of complex systems in ways that spreadsheets just can't do:

* Inventory control and billing
* Human resources
* Blogs
* Ecosystems

For this session we are also going to play as we go, so let's begin by installing and importing the SQLAlchemy library. This provides an interface to Python's sqlite3 module.

### Install and load SQLAlchemy

In [26]:
#!pip3 install sqlalchemy
#!pip install sqlalchemy

In [27]:
import sqlalchemy

In [31]:
# Some info links:

# https://www.datanamic.com/support/lt-dez005-introduction-db-modeling.html
# https://www.datanamic.com/support/database-normalization.html
# https://www.fullstackpython.com/databases.html
# https://www.sqlalchemy.org/
# https://opentextbc.ca/dbdesign01/chapter/chapter-8-entity-relationship-model/

In [32]:
from sqlalchemy import create_engine

In [33]:
engine = create_engine('sqlite:///:memory:', echo=True)

# The Entity Relationship Data Model
------------------------------------------------------------------------------------------------

The entity relationship (ER) model is a commonly used model for defining and developing databases. In the simplest terms, the model defines the things (entities) that are important or interesting within a system or process and the relationships between them.

Entities are _nouns_, and can be physical or logical:

* People - bakers, teachers, students, musicians
* Places - stores, websites, states
* Things - donuts, albums, purchases

Entities are represented as tables within a database.

Entities have properties or attributes which describe them. 

### Exercise

Use the **WWW SQL Designer** at [http://ondras.zarovi.cz/sql/demo/](http://ondras.zarovi.cz/sql/demo/)

In [ ]:
# http://ondras.zarovi.cz/sql/demo/